<a href="https://colab.research.google.com/github/wonjun0804/Crime_Predictor/blob/main/%EC%84%9C%EC%9A%B8%EC%8B%9C_%EC%9E%90%EC%B9%98%EA%B5%AC%EB%B3%84_%ED%8E%B8%EC%9D%98%EC%A0%90%EC%8B%9C%EC%84%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#서울시 휴게음식점 인허가 정보.csv 파일에서 "영업상태명"이 "폐업"이나 "취소/말소/만료/정지/중지"인 데이터는 제거하고, 사업장명에 "cu", "씨유", "gs", "지에스", "세븐일레븐", "이마트24", "미니스톱"이 포함된 편의점만 남긴다음 서울 자치구 25개로 나눠서 업소 수를 합산

import pandas as pd

# 1. CSV 파일 불러오기
df = pd.read_csv("서울시 휴게음식점 인허가 정보.csv", encoding="utf-8")


# 2. 제거할 영업상태명 정의
drop_statuses = ['폐업', '취소/말소/만료/정지/중지']

# 3. 비영업 데이터 제거
df = df[~df['영업상태명'].isin(drop_statuses)].copy()

# 4. 편의점 브랜드 키워드 정의 (모두 소문자로 처리)
keywords = ['cu', '씨유', 'gs', '지에스', '세븐일레븐', '이마트24', '미니스톱']

# 5. 사업장명을 소문자로 변환
df['사업장명_소문자'] = df['사업장명'].astype(str).str.lower()

# 6. 키워드 중 하나라도 포함된 경우만 남기기
df = df[df['사업장명_소문자'].apply(lambda x: any(k in x for k in keywords))].copy()

# 7. 자치구 추출 함수 정의
def extract_gu(address):
    try:
        if pd.notnull(address) and "구" in address:
            return address.split()[1]  # 예: 서울특별시 강남구 → 강남구
        else:
            return None
    except:
        return None

# 8. 도로명주소에서 자치구 추출
df['자치구'] = df['도로명주소'].apply(extract_gu)

# 9. 도로명주소에서 못 뽑은 경우 지번주소에서 추출
df['자치구'] = df['자치구'].fillna(df['지번주소'].apply(extract_gu))

# 10. 자치구별 편의점 수 집계
gu_counts = df['자치구'].value_counts().reset_index()
gu_counts.columns = ['자치구', '편의점수']

# 11. CSV로 저장 (선택)
gu_counts.to_csv("서울시_자치구별_편의점시설.csv", index=False, encoding="utf-8-sig")

<ipython-input-6-1185970362>:6: DtypeWarning: Columns (36,41,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("서울시 휴게음식점 인허가 정보.csv", encoding="utf-8")
